
<h1 style="font-family:verdana;font-size:300%;text-align:center;background-color:#f2f2f2;color:#0d0d0d">AMMI NLP </h1>

<h1 style="font-family:verdana;font-size:180%;text-align:Center;color:#993333"> Lab 3: n-gram models </h1>

In Today's lab we are going to implement a simple language model based on n-gram. n-gram models predicts the next word given previous pre-dermined words/context. For Instance, given a sentence like “when will you come back ___”, the model should predict the next correct word with high probability. n-gram has many other applications. For better understanding have a look at the following resources and also your lecture notes:

- https://web.stanford.edu/~jurafsky/slp3/3.pdf
- http://www.cs.columbia.edu/~mcollins/courses/nlp2011/notes/lm.pdf


**Big thanks to Amr Khalifa (AMMI Tutor 2019-2020) who improved this lab and made it to a Jupyter Notebook!**

In [1]:
import io, sys, math, re
from collections import defaultdict
import numpy as np

In [2]:
# data_loader
def load_data(filename):
    '''
    parameters:
    filename (string): datafile
    
    Returns:+++
    data (list of lists): each list is a sentence of the text 
    vocab (dictionary): {word: no of times it appears in the text}
    '''
    fin = io.open(filename, 'r', encoding='utf-8')
    data = []
    vocab = defaultdict(lambda:0)
    for line in fin:
        sentence = line.split()
        data.append(sentence)
        for word in sentence:
            vocab[word] += 1
    return data, vocab

In [3]:

print("load training set..")
print("\n")
train_data, vocab = load_data("train1.txt")
print(train_data[0])
print("\n")
print("how :",vocab['how'])
print("load validation set")
valid_data, _ = load_data("valid1.txt")


load training set..


['<s>', 'my', 'fathers', "don't", 'speak', 'dutch.', '</s>']


how : 107
load validation set


In [4]:
def remove_rare_words(data, vocab, mincount = 1):
    '''
    Parameters:
    data (list of lists): each list is a sentence of the text 
    vocab (dictionary): {word: no of times it appears in the text}
    mincount(int): the minimum count 
    
    Returns: 
    data_with_unk(list of lists): data after replacing rare words with <unk> token
    '''
    # replace words in data that are not in the vocab 
    # or have a count that is below mincount
    data_with_unk = []

    ## FILL CODE
    for sentence in data:
        s = []
        for word in sentence:
            if word not in vocab or vocab[word] <= mincount:
                s.append('<unk>')
            else:
                s.append(word)
        data_with_unk.append(s)
    
    return data_with_unk

In [5]:
print("remove rare words")
train_data = remove_rare_words(train_data, vocab, mincount = 1)
valid_data = remove_rare_words(valid_data, vocab, mincount = 1)
#train_data
print(train_data[0])


remove rare words
['<s>', 'my', '<unk>', "don't", 'speak', '<unk>', '</s>']


For Example, in bigram we have:

$$P(w_t|w_{t-1}) = \frac{c(w_{t-1}, w_t)}{c(w_{t-1})}$$

Please generalize this to n-gram, n being any positive number. 

In [6]:
def build_ngram(data, n):
    '''
    Parameters:
    data (list of lists): each list is a sentence of the text 
    n (int): size of the n-gram
    
    Returns:
    prob (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    '''
    total_number_words = 0
    counts = defaultdict(lambda: defaultdict(lambda: 0.0))

    for sentence in data:
        sentence = tuple(sentence)
        ## FILL CODE
        # dict can be indexed by tuples
        # store in the same dict all the ngrams
        # by using the context as a key and the word as a value
    
        for i in range(len(sentence)):
            for k in range(n):
                if i < k:
                    break   
                context = sentence[i-k:i]
                #print(context, sentence[i])
                counts[context][sentence[i]] += 1

    prob = defaultdict(lambda: defaultdict(lambda: 0.0))
    # Build the probabilities from the counts
    # Be careful with how you normalize!

    for context in counts.keys():
    ## FILL CODE
        context_den = 0
        
        for w in counts[context].keys():
            context_den += counts[context][w]
        
        for w in counts[context].keys():
            prob[context][w] = counts[context][w]/context_den

    return prob

In [7]:
# RUN TO BUILD NGRAM MODEL

n = 4
print("build ngram model with n = ", n)
model = build_ngram(train_data, n)

build ngram model with n =  4


Here, implement a recursive function over shorter and shorter context to compute a "stupid backoff model". An interpolation model can also be implemented this way.

Stupid backoff:

if $c(w_{t-n+1} ... w_t) > 0:$
$$P_{bo}(w_t|w_{t-n+1} ... w_{t-1}) = \frac{c(w_{t-n+1} ... w_t)}{c(w_{t-n+1} ... w_{t-1})}$$

else backoff to (n-1) gram:
$$ P_{bo}(w_t|w_{t-n+1} ... w_{t-1}) = 0.4 P_{bo}(w_t|w_{t-n+2} ... w_{t-1})$$


In [8]:
def get_prob(model, context, w):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    } 
    context (list of strings): a sentence
    w(string): the word we need to find it's probability given the context
    
    Retunrs:
    prob(float): probability of this word given the context 
    '''

    # code a recursive function over 
    # smaller and smaller context
    # to compute the backoff model
    
    ## FILL CODE
    context = tuple(context)
    #print(model[context][w])
    if model[context][w] > 0.0:
        return model[context][w]
    else:
        return 0.4 * get_prob(model, context[1:], w)

    #return 0.0


$$Perplexity = 2 ^ {-L}$$

$$ L = \frac{1}{M} \sum_{i=1}^N log_2 p(s_i)$$

N = number of sentences

M = total number of words in the test corpus $C$

$p(s_i) = P(w_t|w_{t-1}, ...., w_1)$ ----> probability of the sentence.

$s_1, s_2, ...., s_N \in C$



In [9]:
import math

def perplexity(model, data, n):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    } 
    data (list of lists): each list is a sentence of the text
    n(int): size of the n-gram
    
    Retunrs:
    perp(float): the perplexity of the model 
    '''
    ## FILL CODE
    M = 0
    sp = 0
    
    for sentence in data:
        sentence = tuple(sentence)

        for i in range(1, len(sentence)):
            nc = min(i, n-1)
            context = sentence[i-nc:i]
            sp += math.log(get_prob(model, context, sentence[i]), 2)
            M += 1
        
    L = 1/M * sp
    perp = 2**(-L)

    return perp

In [10]:
# COMPUTE PERPLEXITY ON VALIDATION SET

print("The perplexity is", perplexity(model, valid_data, n=n))

The perplexity is 109.0070144943816


In [58]:
def get_proba_distrib(model, context):
    ## need to get the the words after the context and their probability of appearance
    ## after this context 
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    context (list of strings): the sentence we need to find the words after it and 
    thier probabilites
    
    Retunrs:
    words_and_probs(dic): {word: probability of word given context}
    
    '''
    # code a recursive function over context
    # to find the longest available ngram
    
    ## FILL CODE
#     context = tuple(context)
#     #print('i am here')
#     #defaultdict(lambda: 0.0)
#     words_and_probs = defaultdict(lambda: 0.0)
#     words = [*model[context].keys()]
#     for word in words:
#         words_and_probs[word] = model[context][word]#get_prob(model, context, word)
#     #print(words_and_probs)

#     return words_and_probs

#     print(model.keys())

    if context in model:
        return model[context]
    else:
        return get_proba_distrib(model, context[1:])
    

In [59]:
def generate(model):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    
    Retunrs:
    sentence (list of strings): a sentence sampled according to the language model. 
    

    '''
    # generate a sentence. A sentence starts with a <s> and ends with a </s>
    # Possiblly a use function is:
    # np.random.choice(x, 1, p = y)

    # where x is a list of things to sample from
    # and y is a list of probability (of the same length as x)
    #words_and_probs_ = {}
    sentence = ["<s>"]
    while sentence[-1] != "</s>" and len(sentence)<100:
        ## FILL CODE
        words_and_probs_ = get_proba_distrib(model, tuple(sentence))
        x = list(words_and_probs_.keys())
        y = list(words_and_probs_.values())
        sample = np.random.choice(x, 1, p = y)[0]
        sentence += [sample]
        
    return sentence

In [66]:
# GENERATE A SENTENCE FROM THE MODEL

print("Generated sentence: ", generate(model))

Generated sentence:  ['<s>', 'tom', 'is', 'the', 'one', 'who', 'helped', 'mary', 'do', 'that.', '</s>']


In [70]:
aa_milne_arr = ['pooh', 'rabbit', 'piglet', 'Christopher']
np.random.choice(aa_milne_arr, 1, p=[0.5, 0.1, 0.1, 0.3])

array(['Christopher'], dtype='<U11')

Once you are done implementing the model, evaluation and generation code, you can try changing the value of `n`, and play with a larger training set (`train2.txt` and `valid2.txt`). You can also try to implement an interpolation model.

In [121]:
x = ['an', 'nhv']
tuple(tuple(x))

('an', 'nhv')